<a href="https://colab.research.google.com/github/tommyliphysics/tommyli-ml/blob/main/ai_detector/notebooks/deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Large-language models for AI-detection: deBERTa

In this notebook and the next we'll use the data we explored in the previous notebook to train two large-language models available in the huggingface transformers library. This notebook will look at deBERTa v3, and the next will look at distilBERT, which is a smaller model.

As with training any kind of neural network, when fine-tuning large-language models the learning rate and number of epochs must be carefully chosen to avoid overfitting. I'll first use a train-validation split on the training data to carefully tune the learning rate and monitor the validation accuracy with each epoch, beginning with a learning rate of 10$^{-5}$ and then progressively reducing the learning rate until the validation accuracy ceases to increase. After that, we can use the full training set to train a model with the tuned learning rate schedule.

In [1]:
#pip install --upgrade transformers

In [1]:
#!pip install tf-keras
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [2]:
import pandas as pd

samples = pd.read_csv('https://raw.githubusercontent.com/tommyliphysics/tommyli-ml/main/ai_detector/notebooks/samples.csv')
samples

,text,source,topic,TTV split,label
0,I can't honestly believe that this is a sequel...,imdb,movie review,2.0,0
1,LL Cool J performed much better in this movie ...,imdb,movie review,0.0,0
2,It would be unwise to judge that that either n...,imdb,movie review,-1.0,0
3,20th Century Fox's ROAD HOUSE 1948) is not onl...,imdb,movie review,3.0,0
4,"I am a fan of Jess Franco's bizarre style, and...",imdb,movie review,-1.0,0
...,...,...,...,...,...
24175,The Louisville Cardinals men's soccer team is ...,wikipedia by GPT,Louisville Cardinals men's soccer,4.0,1
24176,"KFC Yum! Center, also known as the Yum! Center...",wikipedia by GPT,KFC Yum! Center,4.0,1
24177,The 2020–21 Louisville Cardinals men's basketb...,wikipedia by GPT,2020–21 Louisville Cardinals men's basketball ...,4.0,1
24178,Conte Forum is a multi-purpose indoor arena lo...,wikipedia by GPT,Conte Forum,4.0,1


In [5]:
train = samples[samples['TTV split'] > 0]
val = samples[samples['TTV split'] == 0]

We will now define functions to import the pre-trained model from the huggingface transformers library.

In [3]:
from transformers import TFDebertaV2ForSequenceClassification, DebertaV2Config, DebertaV2TokenizerFast

def get_tokenizer_model(model_path=None):
    tokenizer = DebertaV2TokenizerFast.from_pretrained('microsoft/deberta-v3-small', truncation=True, model_max_length=512)
    model = TFDebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-small', num_labels=2)
    if model_path is not None:
        tokenizer.save_pretrained(model_path)
        print('Tokenizer saved to /' + model_path)
    return tokenizer,model

In [11]:
tokenizer,model = get_tokenizer_model()

All model checkpoint layers were used when initializing TFDebertaV2ForSequenceClassification.

Some layers of TFDebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['pooler', 'cls_dropout', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We will use keras to fine-tune the pre-trained models on our human and AI-generated texts. Let's define a function that prepares text samples for training, and then prepare the training and validation data.

In [4]:
import tensorflow as tf

def tokenize_encode(text_list):
    return tokenizer(text_list, truncation=True, padding=True)

def create_dataset(samples):
    encodings = tokenize_encode(samples['text'].tolist())
    return tf.data.Dataset.from_tensor_slices((
      dict(encodings),
      samples['label'].tolist()
    )).shuffle(len(samples)).batch(16)

In [10]:
train_dataset = create_dataset(train)
val_dataset = create_dataset(val)

Next we can define our functions to compile and train the model.

In [5]:
import numpy as np

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler

import math

import keras
from keras.callbacks import Callback

def compile_model(model):
    model.compile(optimizer=RMSprop(learning_rate=learning_rate),
                  metrics = ['accuracy'])
    model.config.id2label = {0: 'human', 1: 'AI'}

def fit_model(model, model_path=None):
    history = model.fit(train_dataset,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[LearningRateScheduler(lr_scheduler)],
        validation_data=val_dataset,
        verbose=1)



We can now compile and fine tune the keras model. I'll start training at a learning rate of 10$^{-5}$ then reduce it by a factor of 10 for 3 more epochs.

In [12]:
learning_rate = 1e-5
compile_model(model)

In [ ]:
def lr_scheduler(epoch, lr):
    return learning_rate

In [13]:
epochs = 1
batch_size=128

In [14]:
fit_model(model)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


956/956 [==============================] - 1824s 2s/step - loss: 0.1138 - accuracy: 0.9445 - val_loss: 0.0566 - val_accuracy: 0.9835 - lr: 1.0000e-05


In [15]:
learning_rate = 1e-6

In [16]:
fit_model(model)

956/956 [==============================] - 1761s 2s/step - loss: 0.0043 - accuracy: 0.9988 - val_loss: 0.0413 - val_accuracy: 0.9885 - lr: 1.0000e-06


In [17]:
learning_rate = 1e-7

In [18]:
fit_model(model)

956/956 [==============================] - 1760s 2s/step - loss: 0.0035 - accuracy: 0.9990 - val_loss: 0.0351 - val_accuracy: 0.9906 - lr: 1.0000e-07


In [19]:
learning_rate = 1e-8

In [20]:
fit_model(model)

956/956 [==============================] - 1758s 2s/step - loss: 0.0031 - accuracy: 0.9990 - val_loss: 0.0349 - val_accuracy: 0.9906 - lr: 1.0000e-08


The model has stopped improving so we can improve the learning rate slightly and continue to train for another epoch:

In [21]:
learning_rate = 2e-8
fit_model(model)

956/956 [==============================] - 1759s 2s/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 0.0344 - val_accuracy: 0.9906 - lr: 2.0000e-08


The model still is not changing, so we can improve the learning rate again and continue training for another epoch:

In [22]:
learning_rate = 5e-8
fit_model(model)

956/956 [==============================] - 1757s 2s/step - loss: 0.0030 - accuracy: 0.9992 - val_loss: 0.0353 - val_accuracy: 0.9906 - lr: 5.0000e-08


It seems that we have found the optimal model, so we will use the learning rate scheduling we found to train a new model on the full training set.

In [7]:
tokenizer,model = get_tokenizer_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


tf_model.h5:   0%|          | 0.00/565M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDebertaV2ForSequenceClassification.

Some layers of TFDebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['pooler', 'cls_dropout', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
train = samples[samples['TTV split'] >= 0]
val = samples[samples['TTV split'] == 0]
train_dataset = create_dataset(train)

train_dataset = create_dataset(train)
val_dataset = create_dataset(val)

In [9]:
learning_rates = [1e-5, 1e-6, 1e-7]

def lr_scheduler(epoch, lr):
    return learning_rates[epoch]

In [10]:
learning_rate = 1e-5
compile_model(model)

In [11]:
epochs = 3
batch_size=128
fit_model(model)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


1195/1195 [==============================] - 2268s 2s/step - loss: 0.0707 - accuracy: 0.9694 - val_loss: 0.0260 - val_accuracy: 0.9903 - lr: 1.0000e-05
Epoch 2/3
1195/1195 [==============================] - 2200s 2s/step - loss: 0.0032 - accuracy: 0.9993 - val_loss: 0.0383 - val_accuracy: 0.9882 - lr: 1.0000e-06
Epoch 3/3
1195/1195 [==============================] - 2187s 2s/step - loss: 0.0031 - accuracy: 0.9994 - val_loss: 0.0242 - val_accuracy: 0.9924 - lr: 1.0000e-07


We can now push our model to the hub:

In [13]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
model.push_to_hub('ai-detector-deberta')
tokenizer.push_to_hub('ai-detector-deberta')

tf_model.h5:   0%|          | 0.00/568M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tommyliphys/ai-detector-deberta/commit/91307176a42a7d0f5f60a687c054d15ed505895a', commit_message='Upload tokenizer', commit_description='', oid='91307176a42a7d0f5f60a687c054d15ed505895a', pr_url=None, pr_revision=None, pr_num=None)

The model is now available on the huggingface hub at https://huggingface.co/tommyliphys/ai-detector-deberta.